# Data Cleaning - Tiền xử lý dữ liệu

**Người phụ trách:** Người 2 - Data Cleaning Specialist

## Mục tiêu:
- Tiền xử lý và làm sạch dữ liệu
- Xử lý missing values, duplicates, outliers
- Chuẩn hóa format dữ liệu
- Tạo báo cáo về chất lượng dữ liệu


# 1️⃣ ĐỌC DỮ LIỆU & KHÁM PHÁ BAN ĐẦU

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import warnings
warnings.filterwarnings("ignore")

# Đọc dữ liệu và hiển thị thông tin cơ bản
df = pd.read_csv('../data/foodpanda_orders.csv')
print("Kích thước dữ liệu:", df.shape)
print("\nCác cột:")
print(df.columns.tolist())
display(df.head())




Kích thước dữ liệu: (6000, 20)

Các cột:
['customer_id', 'gender', 'age', 'city', 'signup_date', 'order_id', 'order_date', 'restaurant_name', 'dish_name', 'category', 'quantity', 'price', 'payment_method', 'order_frequency', 'last_order_date', 'loyalty_points', 'churned', 'rating', 'rating_date', 'delivery_status']


,customer_id,gender,age,city,signup_date,order_id,order_date,restaurant_name,dish_name,category,quantity,price,payment_method,order_frequency,last_order_date,loyalty_points,churned,rating,rating_date,delivery_status
0,C5663,Male,Adult,Peshawar,1/14/2024,O9663,8/23/2023,McDonald's,Burger,Italian,5,1478.27,Cash,38,7/19/2025,238,Active,3,10/14/2024,Cancelled
1,C2831,Male,Adult,Multan,7/7/2024,O6831,8/23/2023,KFC,Burger,Italian,3,956.04,Wallet,24,11/25/2024,81,Active,2,8/21/2025,Delayed
2,C2851,Other,Senior,Multan,6/20/2025,O6851,8/23/2023,Pizza Hut,Fries,Italian,2,882.51,Cash,42,5/10/2025,82,Inactive,3,9/19/2024,Delayed
3,C1694,Female,Senior,Peshawar,9/5/2023,O5694,8/23/2023,Subway,Pizza,Dessert,4,231.30,Card,27,7/24/2025,45,Inactive,2,6/29/2025,Delayed
4,C4339,Other,Senior,Lahore,12/29/2023,O8339,8/24/2023,KFC,Sandwich,Dessert,1,1156.69,Cash,35,12/21/2024,418,Inactive,3,3/6/2025,Cancelled


# 2️⃣ THỐNG KÊ BAN ĐẦU VỀ DỮ LIỆU

In [2]:

print("\n--- Thông tin dữ liệu ---")
df.info()
print("\n--- Mô tả dữ liệu số ---")
display(df.describe())



--- Thông tin dữ liệu ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      6000 non-null   object 
 1   gender           6000 non-null   object 
 2   age              6000 non-null   object 
 3   city             6000 non-null   object 
 4   signup_date      6000 non-null   object 
 5   order_id         6000 non-null   object 
 6   order_date       6000 non-null   object 
 7   restaurant_name  6000 non-null   object 
 8   dish_name        6000 non-null   object 
 9   category         6000 non-null   object 
 10  quantity         6000 non-null   int64  
 11  price            6000 non-null   float64
 12  payment_method   6000 non-null   object 
 13  order_frequency  6000 non-null   int64  
 14  last_order_date  6000 non-null   object 
 15  loyalty_points   6000 non-null   int64  
 16  churned          6000 non-null   

,quantity,price,order_frequency,loyalty_points,rating
count,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000
mean,2.991667,800.524880,25.304667,250.173667,2.996833
std,1.414778,404.855029,14.358930,144.608478,1.406113
min,1.000000,100.300000,1.000000,0.000000,1.000000
25%,2.000000,441.997500,13.000000,125.000000,2.000000
50%,3.000000,806.460000,25.000000,249.500000,3.000000
75%,4.000000,1149.737500,37.000000,378.000000,4.000000
max,5.000000,1499.950000,50.000000,500.000000,5.000000


# 3️⃣ XỬ LÝ MISSING VALUES

In [3]:
# Kiểm tra tỉ lệ missing
missing_report = df.isnull().sum().sort_values(ascending=False)
missing_percent = (df.isnull().mean() * 100).round(2)
missing_df = pd.DataFrame({'missing_count': missing_report, 'missing_percent': missing_percent})
display(missing_df[missing_df.missing_count > 0])

# Chiến lược xử lý:
# - Nếu >50% missing: loại cột
# - Numeric: thay bằng median
# - Categorical: thay bằng mode
threshold = 0.5 * len(df)
cols_to_drop = [c for c in df.columns if df[c].isnull().sum() > threshold]
df.drop(columns=cols_to_drop, inplace=True)

for col in df.columns:
    if df[col].isnull().sum() > 0:
        if df[col].dtype in ["int64", "float64"]:
            df[col].fillna(df[col].median(), inplace=True)
        else:
            df[col].fillna(df[col].mode()[0], inplace=True)

print(f"✅ Hoàn thành xử lý missing values. Đã loại {len(cols_to_drop)} cột có quá nhiều missing.")


,missing_count,missing_percent


✅ Hoàn thành xử lý missing values. Đã loại 0 cột có quá nhiều missing.


# 4️⃣ XỬ LÝ DUPLICATES

In [4]:
duplicates = df.duplicated().sum()
print(f"Số bản ghi trùng lặp: {duplicates}")
if duplicates > 0:
    df.drop_duplicates(inplace=True)
    print("→ Đã xóa bản ghi trùng lặp.")
else:
    print("✅ Không có bản ghi trùng lặp.")

Số bản ghi trùng lặp: 0
✅ Không có bản ghi trùng lặp.


# 5️⃣ CHUẨN HÓA KIỂU DỮ LIỆU

In [5]:
# a. Các cột ngày
date_cols = ['signup_date', 'order_date', 'last_order_date', 'rating_date']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=False)

# b. Các cột dạng số (chuyển text chứa số)
for col in df.columns:
    if df[col].dtype == object:
        try:
            df[col] = df[col].str.replace(",", "").astype(float)
        except:
            continue

# c. Chuẩn hóa text
cat_cols = df.select_dtypes(include="object").columns
for col in cat_cols:
    df[col] = df[col].astype(str).str.strip().str.lower()

print("✅ Đã chuẩn hóa format dữ liệu.")

✅ Đã chuẩn hóa format dữ liệu.


# 6️⃣ XỬ LÝ OUTLIERS (IQR)

In [6]:
numeric_cols = df.select_dtypes(include=[np.number]).columns

def remove_outliers_iqr(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR
    before = len(df)
    df = df[(df[col] >= lower) & (df[col] <= upper)]
    removed = before - len(df)
    if removed > 0:
        print(f"→ {col}: loại {removed} outliers.")
    return df

for col in numeric_cols:
    df = remove_outliers_iqr(df, col)

print("✅ Hoàn thành xử lý outliers.")

✅ Hoàn thành xử lý outliers.


# 7️⃣ PHÁT HIỆN & XỬ LÝ DỮ LIỆU KHÔNG NHẤT QUÁN

In [7]:
# Kiểm tra các giá trị bất thường trong categorical columns
for col in cat_cols:
    unique_count = df[col].nunique()
    if unique_count < 20:
        print(f"\nCột {col} ({unique_count} giá trị):")
        print(df[col].value_counts())


Cột gender (3 giá trị):
gender
female    2018
male      2017
other     1965
Name: count, dtype: int64

Cột age (3 giá trị):
age
teenager    2062
adult       1984
senior      1954
Name: count, dtype: int64

Cột city (5 giá trị):
city
multan       1256
lahore       1217
peshawar     1195
islamabad    1187
karachi      1145
Name: count, dtype: int64

Cột restaurant_name (5 giá trị):
restaurant_name
subway         1260
pizza hut      1224
kfc            1224
burger king    1151
mcdonald's     1141
Name: count, dtype: int64

Cột dish_name (5 giá trị):
dish_name
pasta       1262
sandwich    1243
pizza       1200
fries       1152
burger      1143
Name: count, dtype: int64

Cột category (5 giá trị):
category
italian        1236
fast food      1222
continental    1211
chinese        1198
dessert        1133
Name: count, dtype: int64

Cột payment_method (3 giá trị):
payment_method
cash      2039
card      2002
wallet    1959
Name: count, dtype: int64

Cột churned (2 giá trị):
churned
active    

# 8️⃣ TIỀN XỬ LÝ BỔ SUNG

In [8]:
# - Tạo cột "total_value" = quantity * price
# - Tạo cột "customer_tenure" = ngày cuối - ngày signup
df['total_value'] = df['quantity'] * df['price']
df['customer_tenure_days'] = (df['last_order_date'] - df['signup_date']).dt.days

# Loại bỏ giá trị âm hoặc bất hợp lý
df = df[df['customer_tenure_days'] >= 0]


# 9️⃣ TẠO BÁO CÁO CHẤT LƯỢNG DỮ LIỆU

In [9]:
report = pd.DataFrame({
    "column": df.columns,
    "dtype": df.dtypes,
    "missing(%)": [100 * df[c].isnull().mean() for c in df.columns],
    "unique_values": [df[c].nunique() for c in df.columns],
    "min": [df[c].min() if df[c].dtype != 'O' else None for c in df.columns],
    "max": [df[c].max() if df[c].dtype != 'O' else None for c in df.columns]
})

print("\n📊 BÁO CÁO CHẤT LƯỢNG DỮ LIỆU SAU LÀM SẠCH:")
display(report)


📊 BÁO CÁO CHẤT LƯỢNG DỮ LIỆU SAU LÀM SẠCH:


,column,dtype,missing(%),unique_values,min,max
customer_id,customer_id,object,0.0,4533,None,None
gender,gender,object,0.0,3,None,None
age,age,object,0.0,3,None,None
city,city,object,0.0,5,None,None
signup_date,signup_date,datetime64[ns],0.0,694,2023-08-22 00:00:00,2025-08-14 00:00:00
order_id,order_id,object,0.0,4533,None,None
order_date,order_date,datetime64[ns],0.0,729,2023-08-23 00:00:00,2025-08-22 00:00:00
restaurant_name,restaurant_name,object,0.0,5,None,None
dish_name,dish_name,object,0.0,5,None,None
category,category,object,0.0,5,None,None


# 🔟 LƯU DỮ LIỆU ĐÃ LÀM SẠCH

In [10]:
output_path = "../data/foodpanda_orders_cleaned.csv"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df.to_csv(output_path, index=False)
print(f"✅ Đã lưu dữ liệu đã làm sạch tại: {output_path}")


✅ Đã lưu dữ liệu đã làm sạch tại: ../data/foodpanda_orders_cleaned.csv
